In [2]:
import cv2
import os
import pandas as pd
import numpy as np

In [3]:
def hist_intersection(hist1, hist2):
    min = np.minimum(hist1, hist2)
    intersection = np.true_divide(np.sum(min), np.sum(hist2))
    return intersection

In [4]:
def detect_and_compare_all(directory, delimiter=1000, mode = "both"):
    count = 0

    rows_list=[]
    rows_list2=[]
    names = []
    for filename in os.listdir(directory):
        if count >= delimiter:
            break
        if "Devils" in filename or "Pulp" in filename:
            genre = "thriller"
        elif "Move" in filename or "Dance" in filename:
            genre = "dance"
        elif "Arctic" in filename:
            genre = "documentary"

        truth = "../gt_shots/TestSet/" + os.path.splitext(os.path.basename(filename))[0] + ".txt"
        idx = detect_shots_and_compare_for_file(os.path.join(directory, filename), truth, genre, mode)
        if mode != "both":
            rows_list.append(idx)
        else:
            rows_list.append(idx[0])
            rows_list2.append(idx[1])
            
        names.append(filename)

        count += 1

    if mode != "both":
        df1 = pd.DataFrame(rows_list)

        df1.columns = ["hits", "false_hits", "misses", "recall", "precision", "f1"]
        df1.index = names

        #display(df1)
        return df1
        
    else:
        df1 = pd.DataFrame(rows_list)

        df1.columns = ["hits", "false_hits", "misses", "recall", "precision", "f1"]
        df1.index = names

        #display(df1)
        
        df2 = pd.DataFrame(rows_list2)

        df2.columns = ["hits", "false_hits", "misses", "recall", "precision", "f1"]
        df2.index = names

        #display(df2)
        
        return df1, df2

In [5]:
def compute_prec_rec(idx, idx_truth):
    hits = 0
    false_hits = 0
    misses = 0
    
    print("detected: ", idx)
    print("truth: ", idx_truth)
    for i in idx:
        if i in idx_truth:
            hits += 1
        else:
            false_hits += 1

    for i in idx_truth:
        if i not in idx:
            misses += 1

    length = idx_truth.size

    recall = hits / length
    precision = (hits - false_hits*0) / length
    if precision + recall != 0:
        f1 = (2 * precision * recall) / (precision + recall)
    else:
        f1 = 0

    return np.array([hits, false_hits, misses, recall, precision, f1])

In [6]:
def print_rec_prec_array(array):
    print("Hits: " + str(array[0]))
    #print("False Hits: " + str(array[1]))
    print("Misses: " + str(array[2]))
    print("Recall: " + str(array[3]))
    print("Precision: " + str(array[4]))
    print("F1 Measure: " + str(array[5]) + "\n")


In [7]:
def detect_shots_and_compare_for_file(filename, truth_file, genre, mode = "both"):

    if filename.endswith(".mp4"):
        
        with open(truth_file) as f:
            nrs = [[int(x) for x in line.split()] for line in f]
        idx_truth = (np.asarray(nrs)).ravel()
    
    
        print(filename + ":")
        
        if mode == "histogram" or mode == "both":
            idx_hist = find_shots_hist(filename, genre)
            res_hist = compute_prec_rec(idx_hist, idx_truth)
            print("HISTOGRAM: ")
            print_rec_prec_array(res_hist)
            if mode != "both":
                return res_hist
            
        if mode == "pixel" or mode == "both":
            idx_diff = find_shots_pix(filename, genre)
            res_pix = compute_prec_rec(idx_diff, idx_truth)
            print("PIXEL DIFFERENCE: ")
            print_rec_prec_array(res_pix)
            if mode != "both":
                return res_pix    
            
        print("\n")
        
        return res_hist, res_pix
        

In [9]:
#HISTOGRAM VERSION (see lecture 14.2.1.3)
def find_shots_hist(filename, genre, mode="intersection"):
    
    thresh = None
    if genre == "thriller":
        thresh_eucl = 3000
        thresh_inter = 0.9
        
    elif genre == "dance":
        thresh_eucl = 6000
        thresh_inter = 0.85
        
    elif genre == "documentary":
        thresh_eucl = 3000
        thresh_inter = 0.9

        
    if filename.endswith(".mp4"):
        vidcap = cv2.VideoCapture(filename)

        success, image = vidcap.read()
        count = 1

        bins = 256
        
        hist_red_old = None
        hist_green_old = None
        hist_blue_old = None
        diff_list = []
        while success:
            success, image = vidcap.read()
            if not success:
                break

            
            hist_red = cv2.calcHist([image], [0], None, [bins], [0, bins])
            hist_green = cv2.calcHist([image], [1], None, [bins], [0, bins])
            hist_blue = cv2.calcHist([image], [2], None, [bins], [0, bins])

            if hist_red_old is not None:
                
                #EUCLIDEAN DISTANCE VERSION (see lecture 12.3.1)
                if mode == "euclidean":
                    dist_red = np.linalg.norm(hist_red - hist_red_old)
                    dist_green = np.linalg.norm(hist_green - hist_green_old)
                    dist_blue = np.linalg.norm(hist_blue - hist_blue_old)

                    avg_dist = (dist_red + dist_green + dist_blue) / 3
                    diff_list.append(avg_dist)
                    where_hist = np.where(np.asarray(diff_list) > thresh)[0] + 4

                #HISTOGRAM INTERSECTION VERSION (see lecture 12.3.1)
                elif mode == "intersection":
                    inter_red = hist_intersection(hist_red, hist_red_old)
                    inter_green = hist_intersection(hist_green, hist_green_old)
                    inter_blue = hist_intersection(hist_blue, hist_blue_old)

                    diff_list.append((inter_red + inter_green + inter_blue)/3)
                    where_hist = np.where(np.asarray(diff_list) < thresh_inter)[0] + 4

                else:
                    return

            hist_red_old = hist_red
            hist_green_old = hist_green
            hist_blue_old = hist_blue

            count += 1

        #if genre == "dance":
            #print(sorted(np.round(diff_list, 4)))
        


        

        idx_hist = np.concatenate((np.array([1]), where_hist))

        return idx_hist

In [11]:
#PIXEL COMPARISON VERSION (see lecture 14.2.1.2)
def find_shots_pix(filename, genre):
    if filename.endswith(".mp4"):
        vidcap = cv2.VideoCapture(filename)

        success, image = vidcap.read()
        count = 1

        old_img = None
        pixel_diff_idx_list = []
        diff_means = []
        while success:
            success, image = vidcap.read()
            if not success:
                break

            img_size = image.shape[0] * image.shape[1] * image.shape[2]
       
            if old_img is not None:
                diff_img = np.abs(np.subtract(image, old_img))

                diff_mean = np.mean(diff_img)
                diff_means.append(diff_mean)
                diff_counter = np.where(np.asarray(diff_img) > 200)[0].size

                #print("diff counter = ", diff_counter)
                #print("img size = ", img_size)
                if diff_counter > img_size / 2:
                    pixel_diff_idx_list.append(count+4)
                    
                #if diff_mean > 100:
                    #pixel_diff_idx_list.append(count+4)

            old_img = image

            count += 1

        #print(sorted(diff_means))
        idx_diff = np.concatenate((np.array([1]), pixel_diff_idx_list))

        return idx_diff